# 测试stdout输出

你的程序中有个方法会输出到标准输出中（ `sys.stdout` ）。

也就是说它会将文本打印到屏幕上面。 

你想写个测试来证明它，给定一个输入，相应的输出能正常显示出来。

**解决方案**

使用 `unittest.mock` 模块中的 `patch()` 函数， 使用起来非常简单，可以为单个测试模拟 `sys.stdout` 然后回滚， 并且不产生大量的临时变量或在测试用例直接暴露状态变量。

作为一个例子，我们在 mymodule 模块中定义如下一个函数：

In [1]:
%%file mymodule.py

def urlprint(protocol, host, domain):
    url = '{}://{}.{}'.format(protocol, host, domain)
    print(url)

Writing mymodule.py


默认情况下内置的 `print` 函数会将输出发送到 `sys.stdout` 。

为了测试输出真的在那里，你可以使用一个替身对象来模拟它，然后使用断言来确认结果。

使用 `unittest.mock` 模块的 `patch()` 方法可以很方便的在测试运行的上下文中替换对象， 并且当测试完成时候自动返回它们的原有状态。

下面是对 `mymodule` 模块的测试代码：

In [5]:
%%file __init__.py
# for import

Writing __init__.py


In [6]:
from io import StringIO
from unittest import TestCase
from unittest.mock import patch
import mymodule

class TestURLPrint(TestCase):
    def test_url_gets_to_stdout(self):
        protocol = 'http'
        host = 'www'
        domain = 'example.com'
        expected_url = '{}://{}.{}\n'.format(protocol, host, domain)

        with patch('sys.stdout', new=StringIO()) as fake_out:
            mymodule.urlprint(protocol, host, domain)
            self.assertEqual(fake_out.getvalue(), expected_url)

In [10]:
t = TestURLPrint()
t.test_url_gets_to_stdout()

In [9]:
mymodule.urlprint(1,2,3)

1://2.3
